Copyright (c) 2025 Quan-feng WU <wuquanfeng@ihep.ac.cn>

This notebook is released under the [MIT License](https://opensource.org/licenses/MIT).

# Preliminaries

In [ ]:
using Distributed

workers() == [1] || (rmprocs ∘ workers)()

addprocs(Sys.CPU_THREADS, exeflags="--project=$(@__DIR__)")
@info "Added $(nworkers()) workers."

In [ ]:
@everywhere using ArbNumerics
@everywhere using DataInterpolations
@everywhere using DelimitedFiles
@everywhere using DifferentialEquations
@everywhere using Format
@everywhere using SpecialFunctions

@everywhere using JLD2

@everywhere using NaturalUnits

In [ ]:
@everywhere natural_format = "%.2e"
@everywhere float_format = "%.1f"
@everywhere convert_to_ArbFloat = x -> ArbFloat(x; bits=100)

In [ ]:
@everywhere include("tool_script-ArbNumerics_compat.jl")
@everywhere include("tool_script-directories.jl")
@everywhere include("tool_script-geomspace.jl")

@everywhere include("tool_script-N_energy_density.jl")
@everywhere include("tool_script-branching_ratios.jl")
@everywhere include("tool_script-sterile_neutrino_production.jl")

@everywhere include("tool_script-integrals.jl")
@everywhere include("tool_script-parameters.jl")

@info "Loaded all tool scripts."

# Evolution Systems

In [ ]:
function evolution_system_SBBN!(du, u, p, η)
    a = exp(η)
    log_Xₙ, log_ρ_SM_in_EU = u

    Xₙ = exp(log_Xₙ)
    ρ_SM = EU(exp(log_ρ_SM_in_EU), 4)

    T_SM = from_ρ_SM_to_T_SM(ρ_SM)

    x = Q / T_SM
    Γₙ_SM = (255 / τ_n) * (12 + 6 * x + x^2) / x^5

    ### Xₙ evolution ##################################
    d_Xₙ_d_t_SM = -Γₙ_SM * (Xₙ - (1 - Xₙ) * exp(-x))
    d_Xₙ_d_t_decay = -Xₙ / τ_n

    d_Xₙ_d_t = d_Xₙ_d_t_SM + d_Xₙ_d_t_decay
    ##################################################

    ### energy density background and Hubble parameter
    H = sqrt(ρ_SM / (3 * M_Pl^2))

    d_log_ρ_SM_d_η = -4
    ##################################################

    du[1] = d_Xₙ_d_t / (H * Xₙ)
    du[2] = d_log_ρ_SM_d_η
    return du
end

evolution_problem_SBBN = ODEProblem(evolution_system_SBBN!,
    map(log ∘ (v -> EUval(EU, v)),
        [inv(1 + exp(Q / GeV(10))), (π^2 / 30) * N_ρ(GeV(10)) * GeV(10)^4]
    ),
    (0, log(1e7))
)
evolution_solution_SBBN = solve(evolution_problem_SBBN, AutoTsit5(Rosenbrock23()))

a_list_SBBN = geomspace(1, 1e7, 1000)
results_list_SBBN = (evolution_solution_SBBN ∘ log).(a_list_SBBN)
ρ_SM_list_SBBN = map((x -> EU(x, 4)) ∘ exp ∘ last, results_list_SBBN)
T_SM_list_SBBN = from_ρ_SM_to_T_SM.(ρ_SM_list_SBBN)
Xₙ_list_SBBN = (exp ∘ first).(results_list_SBBN)

jldopen(joinpath(dump_data_directory, "dump_data_SBBN.jld2"), "w+") do io
    io["T_SM [GeV]"] = EUval.(GeV, T_SM_list_SBBN)
    io["X_n"] = Xₙ_list_SBBN
end

@info "SBBN scenario done."

In [ ]:
@everywhere function evolution_system!(du, u, p, η)
    a = exp(η)
    log_t_in_EU, log_Xₙ,
        log_ρ_SM_in_EU, log_ρ₄_in_EU, log_ρₓ_in_EU,
        log_n_pion_minus_in_EU, log_n_pion_plus_in_EU,
        log_n_kaon_minus_in_EU, log_n_kaon_zero_long_in_EU = u
    m₄, U_a4_norm, Γ₄, Γₓ = p

    t = EU(exp(log_t_in_EU), -1)
    Xₙ = exp(log_Xₙ)
    ρ_SM = EU(exp(log_ρ_SM_in_EU), 4)
    ρ₄ = EU(exp(log_ρ₄_in_EU < -300 ? -300 : log_ρ₄_in_EU), 4)
    ρₓ = EU(exp(log_ρₓ_in_EU), 4)
    n_pion_minus = EU(exp(log_n_pion_minus_in_EU < -300 ? -300 : log_n_pion_minus_in_EU), 3)
    n_pion_plus = EU(exp(log_n_pion_plus_in_EU < -300 ? -300 : log_n_pion_plus_in_EU), 3)
    n_kaon_minus = EU(exp(log_n_kaon_minus_in_EU < -300 ? -300 : log_n_kaon_minus_in_EU), 3)
    n_kaon_zero_long = EU(exp(log_n_kaon_zero_long_in_EU < -300 ? -300 : log_n_kaon_zero_long_in_EU), 3)

    ρ_tot = ρ_SM + ρ₄ + ρₓ
    T_SM = from_ρ_SM_to_T_SM(ρ_SM)

    this_BKG_coeff = BKG_coeff(T_SM, m₄, U_a4_norm, c_ν_e)

    ρ₄_eq = g₄ * numeric_energy_density_FD(T_SM / m₄) * m₄^4
    T₄ = numeric_Tₘ_FD(ρ₄ / (g₄ * m₄^4)) * m₄
    P₄ = iszero(T₄) ? zero(ρ₄) : g₄ * numeric_pressure_FD(T₄ / m₄) * m₄^4
    n₄ = iszero(T₄) ? ρ₄ / m₄ : numeric_number_density_FD(T₄ / m₄) * m₄^3

    Tₓ = (sqrt ∘ sqrt)(ρₓ / (gₓ * π^2 / 30))

    w₄ = P₄ / ρ₄
    # @show w₄, T₄ / m₄

    γ₄ = ρ₄ / (m₄ * n₄)
    Γ₄ /= γ₄
    Γₓ /= γ₄

    n_γ = 2 * ζ₃ * T_SM^3 / π^2
    n_B = η_B * n_γ

    x = Q / T_SM
    Γₙ_SM = (255 / τ_n) * (12 + 6 * x + x^2) / x^5

    ### Xₙ evolution ##################################
    d_Xₙ_d_t_SM = -Γₙ_SM * (Xₙ - (1 - Xₙ) * exp(-x))
    d_Xₙ_d_t_decay = -Xₙ / τ_n
    d_Xₙ_d_t_pion = (1 - Xₙ) * n_pion_minus * σv_pion_minus_pton(T_SM) - Xₙ * n_pion_plus * σv_pion_plus_ntop(T_SM)
    d_Xₙ_d_t_kaon_minus = (1 - Xₙ) * n_kaon_minus * σv_kaon_minus_pton(T_SM) - Xₙ * n_kaon_minus * σv_kaon_minus_ntop(T_SM)
    d_Xₙ_d_t_kaon_zero_long = (1 - Xₙ) * n_kaon_zero_long * σv_kaon_zero_long_pton(T_SM) - Xₙ * n_kaon_zero_long * σv_kaon_zero_long_ntop(T_SM)

    d_Xₙ_d_t = d_Xₙ_d_t_SM + d_Xₙ_d_t_decay +
                d_Xₙ_d_t_pion + d_Xₙ_d_t_kaon_minus +
                d_Xₙ_d_t_kaon_zero_long
    ##################################################

    ### energy density background and Hubble parameter
    H = sqrt(ρ_tot / (3 * M_Pl^2))

    d_log_t_d_η = inv(H * t)
    d_log_ρ_SM_d_η = -4 - this_BKG_coeff * (ρ₄_eq - ρ₄) / ρ_SM + (Γ₄ - Γₓ) * ρ₄ / (H * ρ_SM)
    d_log_ρ₄_d_η = -3 * (1 + w₄) + this_BKG_coeff * (ρ₄_eq - ρ₄) / ρ₄ - Γ₄ / H + exp(-m₄ / Tₓ) * Γₓ * ρₓ / (H * ρ₄)
    d_log_ρₓ_d_η = -4 + Γₓ * ρ₄ / (H * ρₓ) - exp(-m₄ / Tₓ) * Γₓ / H
    ##################################################

    ### number density of mesons evolution ###########
    this_Br_pion_minus = Br_pion_minus(m₄)
    this_Br_pion_plus = Br_pion_plus(m₄)
    this_Br_kaon_minus = Br_kaon_minus(m₄)
    this_Br_kaon_zero_long = Br_kaon_zero_long(m₄)

    d_n_pion_minus_d_t = n₄ * (Γ₄ - Γₓ) * this_Br_pion_minus - Γ_pion_minus * n_pion_minus -
                            σv_pion_minus_pton(T_SM) * (1 - Xₙ) * n_B * n_pion_minus
    d_n_pion_plus_d_t = n₄ * (Γ₄ - Γₓ) * this_Br_pion_plus - Γ_pion_plus * n_pion_plus -
                            σv_pion_plus_ntop(T_SM) * Xₙ * n_B * n_pion_plus
    d_n_kaon_minus_d_t = n₄ * (Γ₄ - Γₓ) * this_Br_kaon_minus - Γ_kaon_minus * n_kaon_minus -
                            σv_kaon_minus_pton(T_SM) * (1 - Xₙ) * n_B * n_kaon_minus -
                            σv_kaon_minus_ntop(T_SM) * Xₙ * n_B * n_kaon_minus
    d_n_kaon_zero_long_d_t = n₄ * (Γ₄ - Γₓ) * this_Br_kaon_zero_long - Γ_kaon_zero_long * n_kaon_zero_long -
                            σv_kaon_zero_long_pton(T_SM) * (1 - Xₙ) * n_B * n_kaon_zero_long -
                            σv_kaon_zero_long_ntop(T_SM) * Xₙ * n_B * n_kaon_zero_long
    ##################################################

    du[1] = d_log_t_d_η
    du[2] = d_Xₙ_d_t / (H * Xₙ)

    du[3] = d_log_ρ_SM_d_η
    du[4] = d_log_ρ₄_d_η
    du[5] = d_log_ρₓ_d_η

    du[6] = iszero(n_pion_minus) ? 0 : d_n_pion_minus_d_t / (H * n_pion_minus)
    du[7] = iszero(n_pion_plus) ? 0 : d_n_pion_plus_d_t / (H * n_pion_plus)
    du[8] = iszero(n_kaon_minus) ? 0 : d_n_kaon_minus_d_t / (H * n_kaon_minus)
    du[9] = iszero(n_kaon_zero_long) ? 0 : d_n_kaon_zero_long_d_t / (H * n_kaon_zero_long)

    # typeof(u) == Vector{Float64} && typeof(du) == Vector{Float64} && open("tmp.log", "a+") do io
        # write(io, "$(EUval(MeV, T_SM))\n$(EUval(MeV, ρ₄))\n$(log_ρ₄_in_EU)\n$(d_log_ρ₄_d_η)\n\n")
    # end

    return du
end

In [ ]:
@everywhere function magic(m₄, U_a4_norm, Brₓ; num_points=1000)
    a_ini = 1

    this_BKG_coeff_peak = BKG_coeff_peak(m₄, U_a4_norm)
    T_ini = 10 * T_crit(m₄, c_ν_e)
    @assert T_ini ≥ m₄
    # T_ini = this_BKG_coeff_peak ≥ 1 ? max(m₄, T_fo(m₄, U_a4_norm, c_ν_e)) : T_crit(m₄, c_ν_e)
    a_end = T_ini / eV(1)
    # a_end = 1e2

    Γ_ν = G_F^2 * m₄^5 * U_a4_norm^2 / (192 * π^3)
    Γ_SM = Γ_ν / Br_ν(m₄)
    Γₓ = Γ_SM * Brₓ / (1 - Brₓ)
    Γ₄ = Γ_SM + Γₓ
    
    t_ini = 2.25 * NU.s * (inv ∘ sqrt)(N_ρ(T_ini) + g₄ * 7 / 8) * (MeV(1) / T_ini)^2 # Eq. (3.56) of Baumann's Cosmology
    Xₙ_ini = inv(1 + exp(Q / T_ini))
    ρ_SM_ini = (π^2 / 30) * N_ρ(T_ini) * T_ini^4
    ρ₄_ini = 1e-30 * ρ_SM_ini
    # ρ₄_ini = this_BKG_coeff_peak ≥ 1 ? numeric_energy_density_FD(T_ini / m₄) * g₄ * m₄^4 : 1e-30 * ρ_SM_ini
    # n₄_ini = numeric_number_density_FD(T_ini / m₄) * g₄ * m₄^3
    ρₓ_ini = 1e-30 * ρ_SM_ini
    ρ_tot_ini = ρ_SM_ini + ρ₄_ini + ρₓ_ini

    n_γ_ini = 2 * ζ₃ * T_ini^3 / π^2
    n_meson_ini = 1e-20 * n_γ_ini

    evolution_problem = ODEProblem(evolution_system!,
        map(log ∘ (v -> EUval(EU, v)),
            [
                t_ini, Xₙ_ini,
                ρ_SM_ini, ρ₄_ini, ρₓ_ini,
                n_meson_ini, n_meson_ini, n_meson_ini, n_meson_ini
            ]
        ),
        (log(a_ini), log(a_end)),
        (m₄, U_a4_norm, Γ₄, Γₓ)
    )
    # evolution_solution = solve(evolution_problem, AutoTsit5(Rodas4P()))
    evolution_solution = solve(evolution_problem, Rodas5P())
    # evolution_solution = solve(evolution_problem, Rodas4P())
    evolution_solution.retcode == SciMLBase.ReturnCode.Success ||
        @warn "$(m₄), $(U_a4_norm), $(Brₓ): $(evolution_solution.retcode)"

    a_list = geomspace(a_ini, a_end, num_points)
    results_list = (evolution_solution ∘ log).(a_list)

    t_list = map((q -> EU(q, -1)) ∘ exp ∘ (v -> v[1]), results_list)
    Xₙ_list = map(exp ∘ (v -> v[2]), results_list)
    ρ_SM_list = map((q -> EU(q, 4)) ∘ exp ∘ (v -> v[3]), results_list)
    ρ₄_list = map((q -> EU(q, 4)) ∘ exp ∘ (v -> v[4]), results_list)
    ρₓ_list = map((q -> EU(q, 4)) ∘ exp ∘ (v -> v[5]), results_list)
    # ρ_tot_list = map((q -> EU(q, 4)) ∘ exp ∘ (v -> v[3]), results_list)
    # ρ_SM_list = map((q -> EU(q, 4)) ∘ exp ∘ (v -> v[4]), results_list)
    # ρ₄_list = map((q -> EU(q, 4)) ∘ exp ∘ (v -> v[5]), results_list)
    # ρₓ_list = ρ_tot_list .- ρ_SM_list .- ρ₄_list

    ρ_tot_list = ρ_SM_list .+ ρ₄_list .+ ρₓ_list
    T_SM_list = map(from_ρ_SM_to_T_SM, ρ_SM_list)

    ρ_γ_list = π^2 / 30 * 2 * T_SM_list.^4
    ρ_ν_list = ρ_γ_list * 7 / 8
    T_ν_decouple_first_index = findfirst(≤(MeV(0.8)), T_SM_list)
    isnothing(T_ν_decouple_first_index) || for ii ∈ T_ν_decouple_first_index+1:length(T_SM_list)
        ρ_ν_list[ii] = ρ_ν_list[T_ν_decouple_first_index] * (a_list[T_ν_decouple_first_index] / a_list[ii])^4
    end

    return a_list,
        t_list, Xₙ_list,
        ρ_tot_list,
        ρ_SM_list, ρ₄_list, ρₓ_list,
        T_SM_list, ρ_γ_list, ρ_ν_list,
        Γ₄, Γ_SM, Γₓ
end

# Parameter Space Searching

In [ ]:
@everywhere function magic_parameters(param_tuple)
    @assert length(param_tuple) == 3
    # Brₓ, m₄, U_a4_norm = param_tuple
    Brₓ = convert_to_ArbFloat(param_tuple[1])
    m₄ = (EU ∘ convert_to_ArbFloat ∘ EUval)(EU, param_tuple[2])
    U_a4_norm = convert_to_ArbFloat(param_tuple[3])

    a_list,
        t_list, Xₙ_list,
        ρ_tot_list,
        ρ_SM_list, ρ₄_list, ρₓ_list,
        T_SM_list, ρ_γ_list, ρ_ν_list,
        # Γ₄, Γ_SM, Γₓ = magic(m₄, U_a4_norm, Brₓ)
        Γ₄, Γ_SM, Γₓ = magic(m₄, U_a4_norm, Brₓ; num_points=1001)

    log10_Q_over_T_SM_list = log10.(Q ./ T_SM_list)
    perm_indicies = sortperm(log10_Q_over_T_SM_list) 
    Xₙ_vs_lgx_interpolation = LinearInterpolation(Xₙ_list[perm_indicies], log10_Q_over_T_SM_list[perm_indicies]; extrapolation=ExtrapolationType.Extension, cache_parameters=true)
    Xₙ_nuc = (Xₙ_vs_lgx_interpolation ∘ log10)(Q / T_nuc)
    Yₚ = 2 * Xₙ_nuc / (1 - Xₙ_nuc)

    ρₓ_over_ρ_γ_vs_lgx_interpolation = LinearInterpolation((ρₓ_list ./ ρ_γ_list)[perm_indicies], log10_Q_over_T_SM_list[perm_indicies]; extrapolation=ExtrapolationType.Extension, cache_parameters=true)
    ρₓ_over_ρ_γ_nuc = (ρₓ_over_ρ_γ_vs_lgx_interpolation ∘ log10)(Q / last(T_SM_list))
    ΔN_eff = ρₓ_over_ρ_γ_nuc / ((7/8) * cbrt(4/11)^4)

    # jldopen(
    #     joinpath(dump_data_directory,
    #         "dump_data_$(cfmt(natural_format, EUval(GeV, m₄)))GeV_$(cfmt(natural_format, U_a4_norm^2))_$(cfmt(float_format, 100 * Brₓ))percent.jld2"
    #     ), "w+"
    # ) do io
    #     io["a"] = a_list
    #     io["t [GeV^-1]"] = EUval.(GeV, t_list)
    #     io["X_n"] = Xₙ_list
    #     io["rho_SM [GeV^4]"] = EUval.(GeV, ρ_SM_list)
    #     io["rho_gamma [GeV^4]"] = EUval.(GeV, ρ_γ_list)
    #     io["rho_nu [GeV^4]"] = EUval.(GeV, ρ_ν_list)
    #     io["rho_4 [GeV^4]"] = EUval.(GeV, ρ₄_list)
    #     io["rho_X [GeV^4]"] = EUval.(GeV, ρₓ_list)
    #     io["T_SM [GeV]"] = EUval.(GeV, T_SM_list)
    # end

    dump_indices = 1:10:length(a_list)
    jldopen(
        joinpath(dump_data_directory,
            "dump_data_$(cfmt(natural_format, EUval(GeV, m₄)))GeV_$(cfmt(natural_format, U_a4_norm^2))_$(cfmt(float_format, 100 * Brₓ))percent.jld2"
        ), "w+"
    ) do io
        io["a"] = convert.(Float64, a_list[dump_indices])
        io["t [GeV^-1]"] = convert.(Float64, EUval.(GeV, t_list[dump_indices]))
        io["X_n"] = convert.(Float64, Xₙ_list[dump_indices])
        io["rho_SM [GeV^4]"] = convert.(Float64, EUval.(GeV, ρ_SM_list[dump_indices]))
        io["rho_gamma [GeV^4]"] = convert.(Float64, EUval.(GeV, ρ_γ_list[dump_indices]))
        io["rho_nu [GeV^4]"] = convert.(Float64, EUval.(GeV, ρ_ν_list[dump_indices]))
        io["rho_4 [GeV^4]"] = convert.(Float64, EUval.(GeV, ρ₄_list[dump_indices]))
        io["rho_X [GeV^4]"] = convert.(Float64, EUval.(GeV, ρₓ_list[dump_indices]))
        io["T_SM [GeV]"] = convert.(Float64, EUval.(GeV, T_SM_list[dump_indices]))
    end

    return Yₚ, ΔN_eff, Γ_SM
end

In [ ]:
m₄_list = geomspace(GeV(1e-1), GeV(1e1), 50)
U_e4_norm_list = geomspace(1e-9, 1e-2, 50)
Brₓ_list = [0, .1, .5, .9, .99, .999]

output_list = pmap(magic_parameters, Iterators.product(Brₓ_list, m₄_list, U_e4_norm_list))
# output_list = [
#     magic_parameters(param_tuple)
#     for param_tuple ∈ Iterators.product(Brₓ_list, m₄_list, U_e4_norm_list)
# ]

Yₚ_list = first.(output_list)
ΔN_eff_list = getindex.(output_list, 2)
Γ_SM_list = last.(output_list)

jldopen(joinpath(output_data_directory, "results.jld2"), "w+") do io
    io["m4 [GeV]"] = EUval.(GeV, m₄_list)
    io["U_e4_norm"] = U_e4_norm_list
    for (ii, Brₓ) ∈ enumerate(Brₓ_list)
        io["Y_P for Br_x = $(cfmt(float_format, 100 * Brₓ))%"] = convert.(Float64, Yₚ_list[ii, :, :])
        io["Delta_N_eff for Br_x = $(cfmt(float_format, 100 * Brₓ))%"] = convert.(Float64, ΔN_eff_list[ii, :, :])
        io["Gamma_SM [GeV] for Br_x = $(cfmt(float_format, 100 * Brₓ))%"] = convert.(Float64, EUval.(GeV, Γ_SM_list[ii, :, :]))
    end
end
@info "Parameter searching done."

# Collect Data

In [ ]:
jldopen(joinpath(output_data_directory, "dump_data", "collected_dump_data.jld2"), "w+") do dump_jld2
    dump_jld2["columns"] = ["a", "t [GeV^-1]", "X_n", "rho_SM [GeV^4]", "rho_gamma [GeV^4]", "rho_nu [GeV^4]", "rho_4 [GeV^4]", "rho_X [GeV^4]", "T_SM [GeV]"]
    for (m₄, U_e4_norm, Brₓ) ∈ Iterators.product(m₄_list, U_e4_norm_list, Brₓ_list)
        dump_data = load(
            joinpath(output_data_directory, "dump_data",
                "dump_data_$(cfmt(natural_format, EUval(GeV, m₄)))GeV_$(cfmt(natural_format, U_e4_norm^2))_$(cfmt(float_format, 100 * Brₓ))percent.jld2"
            )
        )
        dump_jld2["m_4: $(cfmt(natural_format, EUval(GeV, m₄)))GeV U_e4^2: $(cfmt(natural_format, U_e4_norm^2)) Br_x: $(cfmt(float_format, 100 * Brₓ))%"] =
            hcat(
                dump_data["a"],
                dump_data["t [GeV^-1]"],
                dump_data["X_n"],
                dump_data["rho_SM [GeV^4]"],
                dump_data["rho_gamma [GeV^4]"],
                dump_data["rho_nu [GeV^4]"],
                dump_data["rho_4 [GeV^4]"],
                dump_data["rho_X [GeV^4]"],
                dump_data["T_SM [GeV]"]
            )
    end
end

# Shutdown Julia Workers

In [ ]:
(rmprocs ∘ workers)()